In [1]:
import pandas as pd
import spacy
import re
import tldextract
from sklearn.preprocessing import MultiLabelBinarizer,OneHotEncoder
import tensorflow as tf
import numpy as np

nlp = spacy.load('en_core_web_sm')

STOP_WORDS = nlp.Defaults.stop_words

nRowsRead = 50  # specify 'None' if want to read whole file
postStrings = []
typeStrings = []

2022-09-02 11:36:57.053198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-02 11:36:57.053226: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-02 11:36:58.525145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-02 11:36:58.525165: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-02 11:36:58.525189: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marko-pc): /proc/driver/nvidia/version does not exist


In [2]:
df = pd.read_csv('./input/mbti_1.csv', delimiter=',', nrows=nRowsRead)
df.dataframeName = 'mbti_1.csv'

In [3]:
# making array of array of posts
for person,type in zip(df['posts'],df['type']):
    postStrings.append([type,person.split('|||')])

# removing excess single parentheses

for i in range(0,len(postStrings)):
    postStrings[i][1][0] = postStrings[i][1][0][1:]
    postStrings[i][1][-1] = postStrings[i][1][-1][:-1]

In [4]:
#splitting posts into list of posts
df['posts'] = df['posts'].apply(lambda x: x.split('|||'))

# removing excess single parentheses
def removeParentheses(posts):
    posts[0] = posts[0][1:]
    posts[-1] = posts[-1][:-1]
    return posts
df['posts'] = df['posts'].apply(lambda x: removeParentheses(x))

df.head()

,type,posts
0,INFJ,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,ENTP,[I'm finding the lack of me in these posts ver...
2,INTP,[Good one _____ https://www.youtube.com/wat...
3,INTJ,"[Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,"[You're fired., That's another silly misconcep..."


In [5]:
#label encoding type

def labelEncodeType(type):
  if type == 'ISTJ' or type == 'ISFJ' or type == 'ESTJ' or type == 'ESFJ':
    type = 0 #GUARDIAN
  elif type == 'ISTP' or type == 'ISFP' or type == 'ESTP' or type == 'ESFP':
    type = 1 #ARTISAN
  elif type == 'INFJ' or type == 'INFP' or type == 'ENFP' or type == 'ENFJ':
    type = 2  # IDEALIST
  elif type == 'INTJ' or type == 'INTP' or type == 'ENTP' or type == 'ENTJ':
    type = 3  # RATIONALIST
  return type

df['type'] = df['type'].apply(lambda x: labelEncodeType(x))
df.head()

,type,posts
0,2,"[http://www.youtube.com/watch?v=qsXHcwe3krw, h..."
1,3,[I'm finding the lack of me in these posts ver...
2,3,[Good one _____ https://www.youtube.com/wat...
3,3,"[Dear INTP, I enjoyed our conversation the o..."
4,3,"[You're fired., That's another silly misconcep..."


In [6]:
URL_REGEX = r"""(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))"""
DOMAIN_REGEX = "^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/?\n]+)"
COMMENT_REGEX = r"\[.*?\]"
ELLIPSIS_REGEX = r"^(\ *\.{3}\ *)|(\ *\.{3}\ *)$"

def urlReplace(stringToReplace):
	domain = tldextract.extract(stringToReplace.group())
	return domain.domain + '.' + domain.suffix


**CONSIDER TRYING WITHOUT STOPWORD REMOVAL**

In [7]:
def containsOnlyUrlsOrNumbers(post):
	splits = post.split()
	for str in splits:
		if(not re.search(URL_REGEX,str) and not re.search('^\d+$',str) and not re.search('^\.\.\.+$',str)):
			return 1
	return 0

def containsComments(post):
	splits = post.split('] ')
	for str in splits:
		if(not re.search(COMMENT_REGEX,str)):
			return 1
		return 0

def removeEllipsis(post):
	return re.sub(ELLIPSIS_REGEX,'',post)

def removeWhitespace(post):
	post = " ".join(post.split())
	return post

def removeStopWords(post):
	post = " ".join([t for t in post.split() if t not in STOP_WORDS])
	return post


# def formatPerson(person):
# 	person[1] = list(map(lambda x: removeWhitespace(x), person[1]))
# 	person[1] = [post for post in person[1] if (
# 		containsOnlyUrlsOrNumbers(post) and containsComments(post))]
# 	person[1] = [removeEllipsis(post) for post in person[1]]
# 	person[1] = list(map(lambda x: re.sub(URL_REGEX, urlReplace, x), person[1]))
# 	return person

# postStrings = list(map(formatPerson,postStrings))

def formatPerson(person):
	person = list(map(lambda x: removeWhitespace(x), person))
	person = [post for post in person if (
		containsOnlyUrlsOrNumbers(post) and containsComments(post))]
	person = [removeEllipsis(post) for post in person]
	person = list(map(lambda x: re.sub(URL_REGEX, urlReplace, x), person))
	person = list(map(lambda x: removeStopWords(x),person))
	return person

df['posts']= df['posts'].apply(lambda x:formatPerson(x))

df.head()

,type,posts
0,2,[enfp intj moments youtube.com sportscenter pl...
1,3,"[I'm finding lack posts alarming., Sex boring ..."
2,3,"[Good _____ youtube.com, Of course, I I know; ..."
3,3,"[Dear INTP, I enjoyed conversation day. Esoter..."
4,3,"[You're fired., That's silly misconception. Th..."


In [8]:
#do the nlp tokenization with spaCy

#docs = [[person[0], list(nlp.pipe(person[1], n_process=-1, disable=["tok2vec",
#                         "tagger", "parser", "attribute_ruler", "lemmatizer"]))] for person in postStrings]

def convertToLemmas(posts):
  docs=nlp.pipe(posts,n_process=-1)
  lemmaList = []
  postList = []
  for doc in docs:
    for token in doc:
      lemma = str(token.lemma_)
      if lemma == '-PRON-' or lemma == 'be':
        lemma = token.text
      lemmaList.append(lemma)
    postList.append(" ".join(lemmaList))
    lemmaList= []
  return postList

#for the dataframe:
df['posts']= df['posts'].apply(lambda x:convertToLemmas(x))
df.head()

,type,posts
0,2,[enfp intj moments youtube.com sportscenter pl...
1,3,"[I 'm find lack post alarm ., sex boring it 's..."
2,3,"[good _ _ _ _ _ youtube.com, of course , I I k..."
3,3,"[Dear INTP , I enjoy conversation day . Esoter..."
4,3,"[you 're fire ., that 's silly misconception ...."


In [9]:
#SPLITTING POSTS INTO SEPERATE ROWS
df = df.explode('posts')
df

,type,posts
0,2,enfp intj moments youtube.com sportscenter pla...
0,2,what life - change experience life ?
0,2,youtube.com youtube.com on repeat today .
0,2,May PerC experience immerse you .
0,2,the thing infj friend post facebook commit sui...
...,...,...
49,3,"it know want look right place . perhaps , open..."
49,3,"if stand up , I shoot he . Rap music day shit ..."
49,3,"then work roughly people town , half male . fo..."
49,3,not christian guy New Zealand . I nz either .


In [10]:
df['type'].value_counts()

2    1209
3    1008
1      99
0      50
Name: type, dtype: int64

In [11]:
df0 = df[df['type']==0].sample(40)
df1 = df[df['type']==1].sample(40)
df2 = df[df['type']==2].sample(40)
df3 = df[df['type']==3].sample(40)

dfr = pd.concat([df0,df1,df2,df3])
dfr

,type,posts
48,0,"Unsure completely understand question , simila..."
48,0,"I would not take care , feel like I 'm good li..."
48,0,Math . mainly traditional teaching method do n...
48,0,"not often , I tend lose come thing happen myse..."
48,0,"I start closely watch action , view action eff..."
...,...,...
7,3,"fan idea celestial die essence know , kind sac..."
1,3,Tik Tok great song . as long mental block sing...
1,3,drop.io v1swck0 :D mic close mouth smokin ace ...
32,3,"I greatly appreciate relationship analysis , r..."


In [ ]:
tokenizedPosts = []
personalityTypes = []
tokenDictionary = dict()

for person in docs:
  for post in person[1]:
    personalityTypes.append(person[0])
    tokenizedPosts.append([token.text for token in post])
    for token in post:
      tokenDictionary[token.text]=0

In [ ]:
# df = pd.DataFrame(columns=['type', 'posts'])

# df['type'] = pd.Series(personalityTypes)
# df['posts'] = pd.Series(tokenizedPosts)

In [ ]:
OHEcategories = []
labelIndex = []

OHEcategories = list(tokenDictionary.keys())

for i in range(len(OHEcategories)):
  labelIndex.append((i+1,OHEcategories[i]))

In [ ]:
#oneHotEncode tokens
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(
        tf.constant(OHEcategories), tf.range(len(OHEcategories))),
    default_value=-1)

label_encoder = tf.keras.layers.Lambda(lambda x: table.lookup(tf.constant(x)))

df['posts'] = df['posts'].apply(lambda x: label_encoder(x))

df['posts']
# layer = tf.keras.layers.CategoryEncoding(num_tokens=len(OHEcategories),output_mode="one_hot")

# layer(label_encoder(df['posts']))


# mlb = MultiLabelBinarizer(sparse_output=True)
# new_features = mlb.fit_transform(df['posts'])

# new_features


In [ ]:
#put encoded features into dataframe

# new_features = pd.Series(list(new_features),dtype='b')
# df = pd.concat([df.type,new_features],axis=1)

# split_df = pd.DataFrame(df[0].tolist())
# df = pd.concat([df.type, split_df], axis=1)

In [ ]:
#oneHotEncode labels

ohe = OneHotEncoder(dtype='b')
new_labels = ohe.fit_transform(np.array(df.type).reshape(-1,1)).toarray()

new_labels

In [ ]:
#put encoded labels into dataframe

# new_labels = pd.Series(list(ohe.fit_transform(np.array(df.type).reshape(-1,1)).toarray()),name='type')
# df = pd.concat([new_labels, split_df], axis=1)